In [1]:
# import packages
import pandas as pd
from bt_classes import *
orig_df = pd.read_csv('../res/test_data.csv',index_col=0,parse_dates=True)
my_backtest(orig_df[:500])

Starting Portfolio Value: 100000.00
Final Portfolio Value: 269972.00
Sharpe: 5.45
Max drawdown: 0.92%
Annual rate: 64.96%
